# Ayudantía 03: Decoradores

#### Gabriel Lyon y Pablo Olea

## Decoradores
- ¿Qué son los decoradores?
- ¿Qué recibe como argumento?
- ¿Qué retorna?
- ¿Cuál es la utilidad de un decorador?

## Cómo construir un decorador

### Ejemplo 1

In [2]:
# -------------------
# Función decoradora
# -------------------
def decorador_msjes(fn):
    def msje_con_tags(*args, **kwargs): # Que es *args y **kwargs?
        return "<H>{}</H>".format(fn(*args, **kwargs))
    return decorador_msjes # Aquí se retorna la función como objeto

# -------------------
# Función a decorar
# -------------------
@decorador_msjes
def saludo_miembro(nombre):
    return "Hola {}! Bienvenido(a) de vuelta!".format(nombre)

@decorador_msjes
def msje_advertencia(warning):
    return "Advertencia! : {}".format(warning)

### Cómo construir un decorador con argumentos : Envolturas

### Ejemplo 2

In [3]:
# -------------------
# Función decoradora
# -------------------
def wrapper_mensajes_tag(tag): # Pasamos como argumento el tag que queremos usar
    # Aquí podemos usar los argumentos del wrapper para nuestro decorador
    def decorador_msjes_tag(fn):
        def msje_tags_distintos(*args, **kwargs):
            return "<{0}>{1}</{0}>".format(tag, fn(*args, **kwargs))
        return msje_tags_distintos # Aquí se retorna la función como objeto
    return decorador_msjes_tag # Aquí se devuelve el decorador mismo

# -------------------
# Función a decorar
# -------------------
@wrapper_mensajes_tag("p")
def saludo_miembro(nombre):
    return "Hola {}! Bienvenido(a) de vuelta!".format(nombre)

@wrapper_mensajes_tag("h")
def msje_advertencia(warning):
    return "Advertencia! : {}".format(warning)
saludo_miembro("José")

'<p>Hola José! Bienvenido(a) de vuelta!</p>'

## Actividad 06 2018-1

## Enunciado

Jeff Musk, CEO de la empresa *Programazon*, se fue de viaje y volvió para encontrar que la base de datos de su sistema de ventas había sido modificado por error. Lamentablemente como CEO de la compañia, no programa *software* hace mucho tiempo; por esto, decidió llamar a los estudiantes de Programación Avanzada, pidiendo ayuda. Para que Jeff Reconozca las funciones arregladas deberá modificarlas utilizando **decoradores**. 

### Funciones del programa

#### No debes modificar estas funciones, solo debes decorarlas.



* `procesar(línea)`: retorna un iterable de strings que se encuentran en una línea separados por ";"



* `leer_archivo(filepath)`: es una función generadora que retorna un diccionario de la forma {nombre_producto: precio} por línea de un archivo filepath.


* `cargar_productos(filepath)`: retorna un diccionario con todos los productos disponibles en la base de datos.


* `abonar(monto, saldo)`: dado un monto y un saldo, la función retorna el saldo actualizado según el monto


* `agregar_al_carro(carro, producos_tienda, productos)`: recibe una lista, un diccionario con todos los productos disponibles y un iterable de productos que se quieren agregar, la función verifica que los productos estén disponibles y se puedan agregar al carro de ser así.


* `pagar(carro, productos_tienda, saldo)`: recibe una lista con productos, un diccionario con todos los productos disponibles y un saldo. Verifica que se puedan pagar todos los productos con el saldo. Además, vacía el carro y descuenta el saldo según la compra hecha. Finalmente, retorna una **tupla** con el carro vacío y el nuevo saldo si la compra fue un éxito, de lo contrario, retorna None.

## Programa Principal a decorar

In [4]:
@invertir_string
def procesar(línea):
    return línea.rstrip('\n').split(';')


def leer_archivo(filepath):
    with open(filepath, mode='r', encoding='UTF-8') as archivo:
        for línea in archivo:
            nombre, precio = procesar(línea)
            yield {nombre: int(precio)}
            

def cargar_productos(filepath):
    productos = {}
    for ítem in leer_archivo(filepath):
        productos.update(ítem)
    return productos


def abonar(monto, saldo):
    saldo += monto
    print("Saldo actual: {}".format(saldo))
    return saldo


def agregar_al_carro(carro, producos_tienda, productos):
    for ítem in productos:
        if ítem not in producos_tienda:
            print("El producto '{}' no existe.".format(ítem))
        else:
            print("El producto '{}' ha sido agregado.".format(ítem))
            carro.append(ítem)
    return carro

@verificar_tipos(list, dict, int)
def pagar(carro, productos_tienda, saldo):
    if not carro:
        print("El carro está vacío.")
        return None

    total = sum(productos_tienda[ítem] for ítem in carro)

    if saldo < total:
        print("Falta dinero, amigue.")
        return None

    print("Compra satisfactoria.")
    carro = []
    nuevo_saldo = abonar(-total, saldo)

    return (carro, nuevo_saldo)

NameError: name 'invertir_string' is not defined

## Decoradores

* `invertir_string` Corrige el comportamiento de la función **procesar**. Una forma de hacerlo es invirtiendo el orden del primer elemento que retorna la función decorada.

In [5]:
def invertir_string(func):
    def wrapper (*args, **kwargs):
        str_invertido = func(*args, **kwargs)
        str_invertido[0] = func(*args, **kwargs)[0][::-1]
        return(str_invertido)
    return wrapper



* `verificar_tipos(tipo_1, ..., tipo_i, ..., tipo_n)` Dada una función, se verifica que los argumentos recibidos sean de los tipos indicados en las parámetros del decorador. En caso de que ocurra alguno de los siguientes errores, se debe imprimir un mensaje indicando el error y retornar False.
    * Si algún argumento no es del tipo especificado.
    * Si la cantidad de argumentos ingresados en el decorador no coincide con la cantidad de elementos que recibe la función decorada.

In [39]:
def verificar_tipos(*tipos):
    def funct(func):
        def wrapper (*args, **kwargs):
            if len(tipos) != len(args):
                print ("Largo no igual")
                return False
            for i in range(0, len(args)):
                if tipos[i] != type(args[i]):
                    print("Tipos no coinciden")
                    return False
            return func(*args)
        return wrapper
    return funct

* `registrar`: Cada vez que una función decorada por **registrar** sea ejecutada, se debe guardar en el archivo **registro.txt** un registro con: el nombre de la función a la que se le aplica el decorador y el resultado retornado. Si el archivo ya existe entonces el registro debe agregarse como una nueva línea, **HINT**: __name__ retorna el nombre de una función.

In [40]:
def registrar(func):
    def wrapper(*args, **kwargs):
        with open("registro.txt", "a", encoding='utf-8') as archivo:
            nombre = func.__name__
            res = func(*args, **kwargs)
            if res is False:
                return res
            else:
                archivo.write(f"{nombre},{res}\n")
    return wrapper
    

## Decorando funciones

   * Para no perder ganancias debes registrar en un log de eventos cada vez que una función relacionada con ventas o movimientos de dinero sea ejecutada

   * El sistema no acepta abonos de dinero si es que los montos o saldos ingresados no son números enteros. Por lo tanto, se deben controlar en la función `abonar` que los argumentos sean **enteros**.
   
   * Al momento de pagar se debe verificar que el formato del carro, los productos disponibles y el saldo sean los correctos. El carro debe ser una **lista**, los productos disponibles deben venir en un **diccionario** y el saldo debe ser un **entero**.
   
   * Cada línea de la base de datos debería ser de la forma producto:precio, sin embargo ha quedado de la forma **otcudorp:precio**. Por lo tanto, la función `procesar` debe retornar el nombre del producto de forma correcta.

In [58]:
@invertir_string
def procesar(línea):
    return línea.rstrip('\n').split(';')


def leer_archivo(filepath):
    with open(filepath, mode='r', encoding='UTF-8') as archivo:
        for línea in archivo:
            nombre, precio = procesar(línea)
            yield {nombre: int(precio)}
            

def cargar_productos(filepath):
    productos = {}
    for ítem in leer_archivo(filepath):
        productos.update(ítem)
    return productos

@verificar_tipos(int, int)
def abonar(monto, saldo):
    saldo += monto
    print("Saldo actual: {}".format(saldo))
    return saldo


def agregar_al_carro(carro, producos_tienda, productos):
    for ítem in productos:
        if ítem not in producos_tienda:
            print("El producto '{}' no existe.".format(ítem))
        else:
            print("El producto '{}' ha sido agregado.".format(ítem))
            carro.append(ítem)
    return carro



@verificar_tipos(list, dict, int)
@registrar
def pagar(carro, productos_tienda, saldo):
    if not carro:
        print("El carro está vacío.")
        return None

    total = sum(productos_tienda[ítem] for ítem in carro)

    if saldo < total:
        print("Falta dinero, amigue.")
        return None

    print("Compra satisfactoria.")
    carro = []
    nuevo_saldo = abonar(-total, saldo)

    return (carro, nuevo_saldo)

## Probando los resultados

In [59]:
productos_prograzom = cargar_productos("productos.csv")
print(productos_prograzom)

{'libro': 500, 'disco': 700, 'vinilo': 900, 'radar': 2200, 'limón': 250, 'palta': 300, 'mango': 200, 'papaya': 330, 'arándano': 10, 'damasco': 100, 'naranja': 110, 'manzana': 120, 'plátano': 130, 'durazno': 140, 'orégano': 420, 'cebolla': 100, 'cerveza': 440, 'carpeta': 350, 'cuaderno': 300, 'mochila': 1100, 'zapatos': 1500, 'pantalón': 900, 'celular': 1800, 'iphone8': 2800, 'laptop': 5000, 'laptopconubuntu': 4000, 'macbook': 9900, 'ajedrez': 1000, 'bitcoin': 9000, 'nebcoin': 650}


In [60]:
carro_polea = []
saldo_polea = 0
saldo_polea = abonar(2000, saldo_polea)
productos_a_comprar = ["libro", "disco"]

Saldo actual: 2000


In [61]:
agregar_al_carro(carro_polea, productos_prograzom, productos_a_comprar)
print(carro_polea)
datos_actualizados = pagar(carro_polea, productos_prograzom, saldo_polea)
print(datos_actualizados)

El producto 'libro' ha sido agregado.
El producto 'disco' ha sido agregado.
['libro', 'disco']
Compra satisfactoria.
Saldo actual: 800
None


## Feedback Ayudantía

### Por favor responder el feedback de la ayudantía (Comentarios, sugerencias, opiniones).
https://docs.google.com/forms/d/1rublnCunwYWYe2QxARiND1hS9WJ9jKLE6WJilA7rRvE